In [2]:
# --- IMPORT ---

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np

from layers import *  # From layers.py in same folder

In [3]:
# --- GLOBAL VAR ---

DATA_DIR = '/tmp/data'
MINIBATCH_SIZE = 50
STEPS = 5000

In [4]:
# --- DEFINE CNN ---

mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])
conv1 = conv_layer(x_image, shape=[5, 5, 1, 32])
conv1_pool = max_pool_2x2(conv1)

conv2 = conv_layer(conv1_pool, shape=[5, 5, 32, 64])
conv2_pool = max_pool_2x2(conv2)

conv2_flat = tf.reshape(conv2_pool, [-1, 7*7*64])
full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

keep_prob = tf.placeholder(tf.float32)
full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

y_conv = full_layer(full1_drop, 10)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [5]:
# --- RUN CNN ---

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(STEPS):
        batch = mnist.train.next_batch(MINIBATCH_SIZE)

        if i % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1],
                                                           keep_prob: 1.0})
            print("step {}, training accuracy {}".format(i, train_accuracy))

        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    X = mnist.test.images.reshape(10, 1000, 784)
    Y = mnist.test.labels.reshape(10, 1000, 10)
    test_accuracy = np.mean([sess.run(accuracy, feed_dict={x:X[i], y_:Y[i], keep_prob:1.0}) for i in range(10)])

print("test accuracy: {}".format(test_accuracy))

step 0, training accuracy 0.18000000715255737
step 100, training accuracy 0.8399999737739563
step 200, training accuracy 0.9200000166893005
step 300, training accuracy 0.9200000166893005
step 400, training accuracy 0.8600000143051147
step 500, training accuracy 0.8199999928474426
step 600, training accuracy 0.8600000143051147
step 700, training accuracy 0.8999999761581421
step 800, training accuracy 0.9599999785423279
step 900, training accuracy 0.9599999785423279
step 1000, training accuracy 0.8600000143051147
step 1100, training accuracy 0.9599999785423279
step 1200, training accuracy 0.9800000190734863
step 1300, training accuracy 0.9800000190734863
step 1400, training accuracy 0.9800000190734863
step 1500, training accuracy 0.9599999785423279
step 1600, training accuracy 0.9599999785423279
step 1700, training accuracy 0.9599999785423279
step 1800, training accuracy 1.0
step 1900, training accuracy 0.9800000190734863
step 2000, training accuracy 1.0
step 2100, training accuracy 1.0
